In [ ]:
# !pip install torch torchvision
# !pip install transformers
# !pip install peft
# !pip install bitsandbytes
# !pip install accelerate -U

In [8]:
import json
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
from peft import LoraConfig, get_peft_model

In [9]:
model_name = "microsoft/phi-1_5"
data_path = "../data/Kotlin/train.jsonl"

In [ ]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.float16)

phi_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["fc1", "fc2","Wqkv", "out_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

phi_model = get_peft_model(phi_model, lora_config)
phi_model.print_trainable_parameters()

In [10]:
def load_and_tokenize_dataset(file_path, tokenizer):
    dataset = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            dataset.append(data)

    input_texts = [data["signature"] for data in dataset]
    target_texts = [data["body"] for data in dataset]

    tokenized_inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt")
    tokenized_targets = tokenizer(target_texts, padding=True, truncation=True, return_tensors="pt")

    tokenized_inputs["labels"] = tokenized_targets["input_ids"]

    dataset = Dataset.from_dict({
        "input_ids": tokenized_inputs["input_ids"],
        "attention_mask": tokenized_inputs["attention_mask"],
        "labels": tokenized_inputs["labels"]
    })

    return dataset

In [11]:
phi_tokenizer = AutoTokenizer.from_pretrained(model_name)
phi_tokenizer.pad_token = phi_tokenizer.eos_token

kt_tokenized_inputs = load_and_tokenize_dataset(data_path, phi_tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=30,
    max_steps=-1,
    num_train_epochs=2,
    disable_tqdm=False
)

trainer = Trainer(
    model=phi_model,
    args=training_args,
    train_dataset=kt_tokenized_inputs,
    data_collator=DataCollatorForLanguageModeling(phi_tokenizer, mlm=False)
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./fine_tuned_model")